# Problem definition and constraints

> You are asked to develop a long-term model to predict the power load 1 year ahead. Disregard 2020, 2021, and 2022: use 2019 as test. Another piece of advice from your colleague Marta. The managers at Zap Inc are not AI experts, so they want to know how accurate your model is and why they should trust it. Be sure to answer their concerns in your notebook.

Zap Inc does not provide any technical constraint. We have no indication on the model to use (e.g. whether we should opt for a more interpretable model) or whether inference time matters. No accuracy baseline or target are provided. We do not know of often it will be asked to provide forecasts or how often it will have to be retrained.

The target is to provide a year-ahead forecast of the aggregate powerload: technically speaking, we shall provide a multi-step forecast from a univariate time series (and exogeous factors).

# Modeling strategy

## Data overview and strategy

For a more thorough analysis of the data, see the first notebook. The data contains daily records of power load in Italy, aggregated. We do not know the energy sources, for example what percentage comes from renewables. If we did, we might even forecast those quantities separatedly, as renewables would be more closely tied to weather forecast. We opt not to add external data, even though this will entail higher error when, say, heat waves or rain impact the production quota from solar energy.

## Modeling Strategy

1. Provide baselines with (seasonal) naive methods.
3. Fit a gradient boosted regression tree (GBRT): a commonly used, black-box-ish algorithm that can capture non-linear patterns in the data with little to no preprocessing
4. Fit a harmonic regression: an additive model used in the literature, possibly easier to interpret

Feature extraction will be different for the two models and will be discussed more thoroughly in the corresponding paragraph.

# Data loading

We load the data and split it into two chunks: the training and test (or hold-out) set. The training set comprises data until december 2018, while 2019 data is held out.

In [ ]:
import polars as pl

from powerload import datasets

# fetch the data
powerload = datasets.fetch_powerload(parser="polars")

# save column names
date_col, target_col = powerload.feature_names, powerload.target_names

# generate train and test set
train, test = powerload.data.filter(pl.col(date_col).dt.year().lt(2019)), powerload.data.filter(pl.col(date_col).dt.year().eq(2019))

y_train, y_test = train["load"].to_numpy(), test["load"].to_numpy()

# Model metrics choice

Zap Inc does not provide a KPI metric, so we are just going to minimise the forecast error. Furthermore, given the lack of indications, we are going to weight the errors uniformly. This means that we can rely on `scikit-learn` and other Python packages that provide an interface to find the optimal parameters with a default (and non-customisable) loss function. In other settings, though, we might want to value underestimates more than overestimates, or viceversa. Another common practice is to give a higher weight to the forecast error of the first $n$ months/weeks/days, or use a decay function so that the weight error of the forecast at $t + 365$ would have a smaller impact.

The choice of an error metric is not trivial, though. There are three main "mainstream" candidates:

- Mean absolute error (MAE). Averages out the error of each prediction, computed as ($|y_t - \hat{y_t}|$). It's easy to interpret, as it preserves the scale and has a unit of measurement.
- Mean absolute percentage error (MAPE). It's the MAE, but the error term is divided by $y_t$ to obtain a percentage. Makes it simpler to perform comparisons, however can become tricky to work with if the $y$ is zero.
- (Root) mean squared error (R)MSE. Same as the MAE, only that the error is computed as $({y_t - \hat{y_t}})^2$. This means that bigger errors weight more. Since the error is squared, the MSE can be much greater than the units of measurement. To shrink it, we can take its squared root.

Since we are using the same units - after all, we are comparing different forecasts for the same time series, the MAE seems an adequate and interpretable choice. We can also compute the MAPE, since our datapoints are always greater than zero.

Another technical note. The chosen models (GBRT, linear regression) minimise the mean squared error. Regardless, the coefficients that minimise the MSE are the same that minimise the MAE.

# Model evaluation strategy

To determine the best model to test on the hold-out data, we shall perform cross validation on the training set. In other words, we shall train a model on seven consecutive years worth of data and test it on the eith: as an example, the first "validation fold" will range from 2006 to 2012 and 2013 will be used as validation set, the second fold will range from 2007 to 2013, while 2014 will be used for validation, and so on. The choice of a 7-year interval is somewhat arbitrary: we would like to have 5 to 10 folds to validate our model, and still have enough data for the model to fit.

This strategy enables us to validate our model multiple times and average out the model prediction, to obtain a more robust estimate of the error and how the model might behave in production.

Another note. Time series cross-validation can happen with or without a refit. We choose to refit the model in order to obtain multiple estimates of the coefficients and simulate what might happen in production. Technically, it'd be possible to devise a cross-validation splitter that, for example, could have the model be retrained on the first day of the month; it does not seem unlikely that such long-term predictions would be updated quite often as new data comes in.

In [ ]:
import numpy as np
from sklearn import model_selection

from powerload import naive
from powerload.model_selection import TimeSeriesCrossValidation

TRAINING_WINDOW = 365 * 7
FORECASTING_HORIZON = 365
STRATEGY = "rolling"

rolling_cv = TimeSeriesCrossValidation(train_size=TRAINING_WINDOW, forecasting_horizon=FORECASTING_HORIZON, strategy=STRATEGY)
scores = ["neg_mean_absolute_percentage_error", "neg_mean_absolute_error"]

def cross_val(estimator, X, y=None, scoring=scores, cv=rolling_cv, **kwargs):
    """Cross validation with some default values."""
    # estimators like `naive.*` do not need X, but X and y are needed for scoring.
    if y is None:
        y = X

    return model_selection.cross_validate(
        estimator=estimator,
        X=X,
        y=y,
        scoring=scoring,
        cv=cv,
        n_jobs=-1,
        **kwargs
    )

(num_folds := rolling_cv.get_n_splits(train))

With such validation scheme, we compute the error 7 times - each time on a different year. This should provide us with a better feel of the model performance on unseen data, i.e. its capability to generalise.

# Baseline

Our baseline is made up of two models that belong to the family of the so-called ["naive" models](https://otexts.com/fpp3/simple-methods.html). The forecasts of the first one are always the last value of the training set, while the second always yields the same window of length `seasonal_period`. In plain terms, the first model will return a straight line whose height is the same as the last point that was observed; whereas the second will repeat the last $n$ values observed.

The chosen `seasonal_period` for the naive forecaster is of 7 days and the most reasonable to start with, though the data is also affected by quarterly patterns (e.g. consumption during winter and summer is significantly different).

The choice befalls on these two because they are relatively straightforward to implement and benchmark. In other words, they might make it easier to spot how big of an accuracy improvement other machine-learning models will yield. Other baseline models could be a moving average of the past `n` observations, or additive models that use the trend and weekly (and possibly quarterly) seasonal patterns as components.

In [ ]:
nv = naive.NaiveForecaster()
snv = naive.SeasonalNaiveForecaster(seasonal_period=7)

## Baseline evaluation

In [ ]:
naive_results = cross_val(nv, X=train["load"])
snaive_results = cross_val(snv, X=train["load"])


def get_average_error(cv_results):
    """Returns average and standard deviation from an array of error metrics."""
    return {
        key: (( x_ := np.abs(metrics)).mean().round(3), x_.std().round(3))
        for key, metrics in cv_results.items()
        if key.startswith("test_")
    }


get_average_error(naive_results), get_average_error(snaive_results)

The interpretation of absolute errors is straightforward: the naive model (the first on the) has an error of approximately 16% (with a variation of ~2.5%). On average, this translates of a prediction off by 130kGHW (+/-131kGWH).

The naive seasonal model's error is comparable: slightly above 15%, though with greater variance (5%), or 135kGHW (+/- 50kGWH). The greater variation signifies that there are folds where the model performs worse on the training data. This degradation is easy to explain: this naive model only contains a seasonal term, while we saw that the data displays a declining trend. Our model does not take this feature into account, so its forecast cannot decrease in time. Besides, the seasonal naive disregards other seasonal patterns.

# Gradient Boosted Regression Tree

We first train a gradient boosted regression tree (GBRT) method as it is widely used and requires little feature engineering by virtue of being a non-linear model. GBRT models performance is almost always superior both on tabular data (see [here](https://arxiv.org/abs/2207.08815)) and time-series data as well (see [here](https://arxiv.org/abs/2101.02118)). So far, GBRT models appear to best even deep learning models, with some notable exceptions (see [this](https://arxiv.org/abs/2305.14406) paper by Zalando). Since we do not have any constraints in terms of interpretability, we start with a GBRT tree.

## Feature engineering

While GBRT does not generally require extensive feature engineering, we want to extract datetime components from the `date` column. This means adding numeric variables to denote the year, month and day of the week. We add a categorical variable for holidays (while an indicator variable to denote weekends was found not to improve the error).

In [ ]:
from powerload import preprocessing

train_gbrt = (
    train
    .pipe(preprocessing.extract_datetime_features, "date", ["year", "month", "weekday"])
    #.pipe(preprocessing.add_weekends, "date")
    .pipe(preprocessing.add_holidays, "date", "IT")
    .select(pl.all().exclude("date", "load"))
)

train_gbrt.head()

Handling of categorical variables (e.g. holidays) is quite simple. As per the docs of the `HistGradientBoostingRegressor` (bold mine):

> For datasets with categorical features, using the **native categorical support is often better than relying on one-hot encoding (OneHotEncoder)**, because one-hot encoding requires more tree depth to achieve equivalent splits. It is also usually **better to rely on the native categorical support rather than to treat categorical features as continuous (ordinal) [i.e., using OrdinalEncoder]**, which happens for ordinal-encoded categorical data, since categories are nominal quantities where order does not matter.

## Modelling pipeline

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline

# store the index of the categorical column
categorical_cols = train_gbrt.select(pl.col(pl.Categorical)).columns
categorical_cols_idx = [train_gbrt.find_idx_by_name(col) for col in categorical_cols]

# get a list of the categories
categories = [train_gbrt[col].cat.get_categories().to_list() for col in categorical_cols]

ordinal_encoder = OrdinalEncoder(
    categories=categories, # categories are required as a (n_features, ) shaped array
)

transformer_gbrt = ColumnTransformer(
    transformers=[
            ("holiday_encoder", ordinal_encoder, categorical_cols_idx)
        ],
        remainder="passthrough",
)

model_gbrt = HistGradientBoostingRegressor(
                categorical_features=categorical_cols_idx,
                random_state=42
            )

pipeline_gbrt = Pipeline(
    steps = [
        ("preprocessing", transformer_gbrt),
        ("gbrt", model_gbrt)
    ]
)

## Model evaluation

In [ ]:
results_gbrt = cross_val(pipeline_gbrt, train_gbrt.to_numpy(), y_train, cv=rolling_cv)

get_average_error(results_gbrt)

The GBRT model returns an error that is more than halved. It averages out at 7.4%, though the standard deviation is higher than 5%. In absolute terms, the prediction, on average, is off by almost 55kGHW (+/- 30kGHW).

This model is also quite simple, but its interpretation requires more effort. We cannot simply inspect the coefficients. However, we could display partial dependence or individual conditional expectation (ICE) plots [ref](https://scikit-learn.org/stable/auto_examples/inspection/plot_partial_dependence.html#sphx-glr-auto-examples-inspection-plot-partial-dependence-py) to "unbox" predictions from the GBRT. Another, although more complex, alternative, is to use [Shapley values](https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Basic%20SHAP%20Interaction%20Value%20Example%20in%20XGBoost.html).

# Harmonic regression

We have outlined that the data features a linear trend; in other words, the power load series is a function of time. We can represent this with a `time` column that is basically a sequence that is 0 for the first observation and increments by 1 every step. In this way, time is represented as a step function, i.e. a line that looks like a staircase.

This representation is intuitive, but its power is limited. In the forecasting literature, power series are often successfully estimated using the so called fourier (or harmonic) regression. For example, in [this paper](https://www.sciencedirect.com/science/article/pii/S2211467X20300778), the authors managed to achieve a ~3.5% error with this method when forecasting power demand from between 2012 2017 in Turkey. The authors explain that this outcome is due to Turkish households use electricity mostly for illumination purposes, and that "in regions where electricity is used for heating, the modulated Fourier Series Expansion is not expected to achieve satisfactory performance" due to climate conditions.

But what is a harmonic regression? Simply put, it's a model with features that are a function of sine and cosine. Sine and cosine are smooth (and non-monotonic) functions that allow us to capture non-linear effects. The idea behind this modeling strategy is that sine and cosine waves add up to build the "wave" pattern of the target series. For another explanation, please see [the reference book](https://otexts.com/fpp3/useful-predictors.html#fourier-series) by Rob J Hyndman and George Athanasopoulos.

![source: TeX StackExchange](https://i.stack.imgur.com/fxG8U.png)

The tecnical formulation of such model is the following:

$$
\hat{x}(t) = \mu + \delta t + \sum_{i=1}^K\left[\alpha_i\sin(2\pi\frac{i}{periodicity}t) + \beta_i\cos(2\pi\frac{i}{periodicity}t)\right]
$$

Where $\mu$ is the intercept, $t$ is the time step, and inside the summation we find $K$ the periodic components. Hyndman and Athanasopoulos recommend $K$ to be no greater than the periodicity. Periodicity, int this case, is the seasonal period of the series. We can have as many periodic "groups" of harmonic components as we wish. In our case, we could try 7 and 365, i.e. weekly and annual periodicity.

## Feature Engineering

Creating these harmonic features is quite easy, but we leave the implementation to the dedicated `powerload.preprocessing` module.

One more thing. Since the model will be a linear regression, we should treat the `holiday` column differently. We cannot use a ordinal encoder, because encoding holidays as integers implies cardinality as well - in other words, it binds the values to a numerical sequence of evenly spaced points, which is an assumption we cannot make (technically speaking, we could move this critique to the GBRT model as well, since we passed the `year`, `month` and `weekday` features as integer variables - though this is arguably less problematic, given the non-parametric non-linear nature of the model).

We could use a different strategy, named one-hot encoding, that turning every category in the `holiday` column into its own column. Every observation in the new column is assigned a zero except when the holiday falls on the correct date. This approach works, except when the resulting dataset is "sparse", i.e. contains a lot of zeros. This will be our case: since each holiday occurs once a year, and our training splits are 7 years long, we would only have 7 nonzero rows out of more than 2500.

For this reason, it is best to turn the column into an indicator variable, i.e. a variable that takes value `1` on holidays and `0` otherwise. Keep in mind that this might cause problems when a holiday falls on a weekend.

In [ ]:
train_harmonic = (
    train
    .pipe(preprocessing.add_time_steps)
    .pipe(preprocessing.add_holidays, "date", "IT")
    .with_columns(pl.when(pl.col("holiday").eq("No")).then(0).otherwise(1).alias("is_holiday"))
    # .pipe(preprocessing.add_weekends, "date")
    .pipe(preprocessing.add_fourier_terms, 3, 7)
    .pipe(preprocessing.add_fourier_terms, 20, 365)
    .select(pl.all().exclude("date", "load", "holiday"))
)

train_harmonic.head()

## Model pipeline

The resulting design matrix cannot be used directly to fit a linear model. Since we are going to fit a linear regression, we'd benefit from bringing all the variables to the same scale. Since the harmonic components are bound to `[-1, 1]`, we standardise the `time` as well as the target columns to the same scale. In this way, the resulting model coefficients will be comparable.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler

transformer_harmonic = ColumnTransformer(transformers=[
    ("scaling", StandardScaler(), (1, )),
    ],
    remainder="passthrough"
)

model_harmonic = TransformedTargetRegressor(regressor=LinearRegression(fit_intercept=True), transformer=StandardScaler())

pipeline_harmonic = Pipeline(
    steps=[
        ("standardise", transformer_harmonic),
        ("harmonic_linreg", model_harmonic)
    ]
)

## Model scoring

In [ ]:
results_harmonic = cross_val(
    pipeline_harmonic,
    train_harmonic.to_numpy(),
    y_train,
    cv=rolling_cv,
    return_estimator=True
)

get_average_error(results_harmonic)

The average error is 5.4% (39kGHW) +/-3% (15kGHW).

## Model inspection

The advantage of a linear model is that it becomes more straighforward to interpret the coefficients. On the other hand, this fourier regression has almost 50 coefficients and the meaning of each sine/cosine pair is a bit unintuitive.

In [ ]:
_coefs_harmonic = np.vstack([
        np.hstack([(reg := pipe["harmonic_linreg"].regressor_).intercept_, reg.coef_]).reshape(1, -1)
        for pipe
        in results_harmonic["estimator"]
])

coefs_harmonic = pl.DataFrame(
    data = [
        ["intercept"] + train_harmonic.columns,
        _coefs_harmonic.mean(axis=0),
        _coefs_harmonic.std(axis=0)
    ],
    schema=["param", "mean", "std"]
)

We can still draw a histogram of these coefficients to see which ones have the biggest effect on the data.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 20))

_ = ax.barh(
    y=coefs_harmonic["param"].to_numpy()[::-1],
    width=coefs_harmonic["mean"].to_numpy()[::-1],
    xerr=coefs_harmonic["std"].to_numpy()[::-1],
    ecolor="red",
    error_kw=dict(elinewidth=3),
)

_ = ax.axvline(x=0, c="black")

The graph above displays the average coefficient from the different estimates across cross-validation. Errorbars are also draw using the standard deviation (though this is not the correct way of estimating the confidence intervals for the coefficients, it can provide an intuition for which parameters should be investigated).

At first glance, we can see that most of these coefficients are different from zero. Most notably, the holiday dummy variable appears to be insignificant. We expected the `time` coefficient to be negative, as there is a declining trend in the data. This, combined with the retrains, also causes the high uncertainty around the intercept.

Almost all weekly harmonic component have a sizeable effect. Weekly effects have the strongest effect. Yearly components are much harder to reason about - technically, there could have been almost 180 for each of sine and cosine, rendering interpretation of each one basically impossible. Perhaps different seasonal patterns could be used, such as monthly (T=28) or quarterly (T=~120).

# In-sample predictions

The `cross_val_predict` interface in scikit-learn does not allow inference with window-based splits (see [here](https://stackoverflow.com/a/43279634/12445701)), so for the moment we shall just inspect in-sample errors after fitting the model to the whole training data.

## Harmonic Regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

_ = pipeline_harmonic.fit(train_harmonic.to_numpy(), y_train)

insample_pred_harmonic = pipeline_harmonic.predict(train_harmonic.to_numpy())

print(f"Harmonic insample mean absolute error: {mean_absolute_error(y_train, insample_pred_harmonic):,.0f} GHW ({mean_absolute_percentage_error(y_train, insample_pred_harmonic):.1%})")

We report the error just for informational purposes. Minimising the in-sample error would yield to overfitting; on the other hand, it's helpful to see whether in-sample and validation error are close. Conversely, if in-sample error were significantly lower, it might hint that our model overfit.

However, if we plot the predictions, we find out that the model systematically overestimates the decrease in demand on holidays. This is due to its linear and additive specification. We also see that the model underestimates surges.

If we look at the residuals (i.e., the difference between the predictions and the true values), we can confidently say there is information left behind. The residuals display non-stationary behavior and are serially correlated. Errors appear to be normally distributed, though the distribution is uneven and skewed. The is still signal to extract - possibly from autoregressive components, but perhaps from modeling the error as a ARIMA process.

In [ ]:
from powerload.diagnostics import plot_predictions

plot_predictions(y_train, insample_pred_harmonic, dates=train["date"], suptitle="Harmonic regression in-sample predictions")

## GBRT

On the other hand, if we inspect the in-sample error in the GBRT model, we notice that it is significantly lower than the cross-validation one - perhaps because it was trained on much more data (almost twice as many observations).

The non-linear nature of the gradient-boosted trees, though, meant that the model does not under-estimate power supply as the harmonic model does. The model under-estimates the extreme peaks, but less than the linear model.

In [ ]:
_ = pipeline_gbrt.fit(train_gbrt.to_numpy(), y_train)
insample_pred_gbrt = pipeline_gbrt.predict(train_gbrt.to_numpy()) 

print(f"GBRT insample error: {mean_absolute_error(y_train, insample_pred_gbrt):,.0f} GHW ({mean_absolute_percentage_error(y_train, insample_pred_gbrt):.1%})")

In [ ]:
plot_predictions(y_train, insample_pred_gbrt, dates=train["date"], suptitle="GBRT in-sample predictions")

This suggests that we try different validation strategies, perhaps with longer but more frequent training windows (e.g. monthly retraining of windows of 10+ years). Still, the residuals display non-stationary behavior and, though normally distributed, appear skewed. Autocorrelation warrants more inspection: the model might benefit from the inclusion of lagged terms, though this poses challenges of its own in inference.

# Performance on the test set

## Harmonic Regression

Finally, we perform data preprocessing on the test set and predict the final result. The performance is abysmal!

In [ ]:
test_harmonic = (
    test
    .pipe(preprocessing.add_time_steps)
    .pipe(preprocessing.add_holidays, "date", "IT")
    .with_columns(pl.when(pl.col("holiday").eq("No")).then(0).otherwise(1).alias("is_holiday"))
    .pipe(preprocessing.add_fourier_terms, 3, 7)
    .pipe(preprocessing.add_fourier_terms, 20, 365)
    .select(pl.all().exclude("date", "load", "holiday"))
)

test_pred_harmonic = pipeline_harmonic.predict(test_harmonic.to_numpy())

print(f"Harmonic test error: {mean_absolute_error(y_test, test_pred_harmonic):,.0f} GHW ({mean_absolute_percentage_error(y_test, test_pred_harmonic):.1%})")

If we inspect the residuals, we notice a huge difference between the means of the forecast and the actual values. We are confident this is due to the harmonic model being trained on a training dataset that is too wide: the intercept coefficient is too big, while the negative coefficient of the `time` column makes the data decrease too slowly. The predictions seem lagged with respect to the actual data and shocks are not learnt properly.

Besides, the residuals clearly display a seasonal pattern. The ACF plot reveals there is still influence from the seventh lag, which might suggest we did not properly model the weekly seasonality.

In [ ]:
plot_predictions(y_test, test_pred_harmonic, dates=test["date"], suptitle="Harmonic regression prediction on the training set")

## GBRT

As a comparison, we also check out the performance of the gradient boosted tree. The error is significantly smaller and similar to the in-sample/validation error.

In [ ]:
test_gbrt = (
    test
    .pipe(preprocessing.extract_datetime_features, "date", ["year", "month", "weekday"])
    .pipe(preprocessing.add_holidays, "date", "IT")
    .select(pl.all().exclude("date", "load"))
)

test_pred_gbrt = pipeline_gbrt.predict(test_gbrt.to_numpy())

print(f"Harmonic test error: {mean_absolute_error(y_test, test_pred_gbrt):,.0f} GHW ({mean_absolute_percentage_error(y_test, test_pred_gbrt):.1%})")

We see that the model fails to take into account the peaks in power supply around July. Part of this error could possibly be improved with weather data. We see that weekly seasonal patterns are learned correctly and there are no problems with drift (which was the biggest error source for the harmonic regression). The model fails to learn the patterns around Christmas, but the residuals surely do not display seasonal patterns. Perhaps the autocorrelation could be resolved by including autoregressive terms.

In [ ]:
plot_predictions(y_test, test_pred_gbrt, dates=test["date"], suptitle="GBRT predictions on the test set")

# Further directions of analysis

* Our validation strategy should be improved. It seems unrealistic to assume that Zap Inc would weight errors uniformly and would definitely retrain the model as data comes in, so we might want to validate a model on (say) monthly retrains and use a weighted error metric. In this way, we might see whether the model fails to achieve an accurate prediction in the short term (~1 month). Besides, the current cross-validation has forecasts start with the beginning of every new year, so the first month we forecast is always January. We might find new insights if we start training from different points in time. A validation strategy closer to production might help us better inform our model.

* We might want to quantify the uncertainty of our predictions. GBRT can also be fit minimising quantile losses; we can minimise the 10th and 90th quantile retrieve confidence intervals for our model (e.g. [here](https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-quantile-py)). This can be simpler than [using MAPIE](https://mapie.readthedocs.io/en/latest/examples_regression/4-tutorials/plot_ts-tutorial.html).

* Adding lagged terms to the GBRT could improve performance. Creating lagged terms introduces missing values, which GBRT can handle natively. On the other hand, adding lagged terms to the harmonic model won't be as straightforward, though it could simply be enough to drop missing values since we would be losing a few rows over seven years worth of observations. If the model were to be retrained every week or month, this should not pose a problem. On the other hand, lagged terms force us to rethink how we obtain predictions (see [here](https://skforecast.org/0.9.1/introduction-forecasting/introduction-forecasting#multi-step-forecasting) for an introduction on single-step vs multi-step forecasting).